# AWS Lambda

<img src="../_assets/aws_service_icons/lambda.svg" width="80" alt="AWS Lambda">

## Goals
- Understand what **AWS Lambda** is
- Know what it’s practically used for
- See a simple **AWS SDK pseudo-code** example (not executed)


## What is AWS Lambda?

**AWS Lambda** is a **serverless compute** service that runs your code in response to **events** and automatically manages the underlying servers.

You deploy a **function** (code + configuration). Lambda then:
- **Invokes** it when triggered (e.g., by API Gateway, EventBridge, S3, SQS, DynamoDB Streams, etc.)
- **Scales** the number of concurrent executions up/down automatically
- **Charges** primarily for invocations and execution time (pay-per-use)

Key concepts:
- **Handler**: entry point function (e.g., `app.handler`)
- **Runtime**: language environment (Python/Node/Java/.NET/Go, or a container image)
- **Execution role (IAM)**: permissions the function has when calling other AWS services
- **Timeout / memory**: bounds and resources per invocation (more memory also increases CPU)


## What is Lambda used for (practically)?

Lambda is commonly used for **event-driven** and **on-demand** compute where you don’t want to manage servers.

Typical use cases:
- **API backends** (API Gateway → Lambda) for lightweight endpoints
- **Async jobs** (SQS → Lambda) such as sending emails, resizing images, or enriching data
- **Scheduled tasks** (EventBridge cron) for periodic housekeeping or data sync
- **Event processing** (S3/DynamoDB Streams/Kinesis) for pipelines and notifications
- **Automation** (glue code) to react to infrastructure/app events
- **Lightweight inference / scoring** for small models (where latency and package size fit)


## Using Lambda with the AWS SDK (pseudo-code)

Below is a minimal, **non-executable** sketch of creating and invoking a Lambda function using an AWS SDK (example shown in Python with `boto3`).

Notes:
- In real projects, avoid hardcoding secrets; prefer **IAM roles** + **Secrets Manager/SSM**.
- Creating AWS resources can incur cost; plan cleanup.
- A Lambda needs an **execution role** with permissions for what it will access.

```python
# PSEUDO-CODE (do not run)

import boto3
import json

region = "us-east-1"
lambda_ = boto3.client("lambda", region_name=region)

function_name = "demo-hello-lambda"
role_arn = "arn:aws:iam::<account-id>:role/<lambda-execution-role>"

# Your function code is typically packaged as a ZIP (or a container image).
# Option A: send ZIP bytes directly.
zip_bytes = b"<zipped function code bytes>"

lambda_.create_function(
    FunctionName=function_name,
    Runtime="python3.12",
    Role=role_arn,
    Handler="app.handler",  # file: app.py, function: handler(event, context)
    Code={"ZipFile": zip_bytes},
    Timeout=10,
    MemorySize=128,
    Publish=True,
)

# Invoke the function synchronously and read the response payload.
event = {"name": "Ada"}
resp = lambda_.invoke(
    FunctionName=function_name,
    InvocationType="RequestResponse",
    Payload=json.dumps(event).encode("utf-8"),
)

payload_text = resp["Payload"].read().decode("utf-8")
print(payload_text)

# Redeploy by updating code.
lambda_.update_function_code(
    FunctionName=function_name,
    ZipFile=b"<new zip bytes>",
    Publish=True,
)

# Clean up (optional):
# lambda_.delete_function(FunctionName=function_name)
```


## Pitfalls & quick tips

- Expect **cold starts** (especially with heavy dependencies or VPC networking).
- Make handlers **idempotent**; many event sources retry on failures.
- Tune **timeout/memory**; memory also scales CPU and can reduce runtime.
- Keep IAM permissions **least-privilege**; separate roles per function where practical.
- Use CloudWatch Logs + metrics (and optionally X-Ray) for **observability**.


## References
- AWS Docs: AWS Lambda (concepts, quotas, pricing)
- AWS Docs: Invoking Lambda functions
- AWS SDK Docs: `boto3` Lambda client reference
